In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

# JS Visualization in Python

Along the spectrum of libraries ranging from pure D3.js to high level Python packages like Pandas, there are a number of JavaScript wrappers that attempt to provide the best JS visualization functionality for use with Python code and data. Usually, this comes at the expense of customizability and low-level control.

This notebook will demo a popular mapping library called Folium, which wraps around Leaflet for interactive mapping functionality. Other Pyhthon visualization libraries also support JavaScript, including:

* [Bokeh](http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html#userguide-interaction-actions-widget-callbacks)
* [Plotly](https://plot.ly/python/#chart-events)

## JS Callbacks in Bokeh

Examples below are taken from the official doc:
http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

In [ ]:
from bokeh.layouts import column
from bokeh.models import *
from bokeh.plotting import Figure, output_file, show
from bokeh.io import output_notebook, reset_output
from random import random

reset_output()
output_notebook()

### Widget callbacks

In [ ]:
x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = Figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    source.trigger('change');
""")

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
# slider = Slider(start=0.1, end=4, value=1, step=.1, title="power", callback=callback)
slider.js_on_change('value', callback)

layout = column(slider, plot)

show(layout)

### Tool callbacks

In [ ]:
source = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[]))

callback = CustomJS(args=dict(source=source), code="""
        // get data source from Callback args
        var data = source.data;

        /// get BoxSelectTool dimensions from cb_data parameter of Callback
        var geometry = cb_data['geometry'];

        /// calculate Rect attributes
        var width = geometry['x1'] - geometry['x0'];
        var height = geometry['y1'] - geometry['y0'];
        var x = geometry['x0'] + width/2;
        var y = geometry['y0'] + height/2;

        /// update data source with new Rect attributes
        data['x'].push(x);
        data['y'].push(y);
        data['width'].push(width);
        data['height'].push(height);

        // trigger update of data source
        source.trigger('change');
    """)

box_select = BoxSelectTool(callback=callback)

p = Figure(plot_width=400,
            plot_height=400,
            tools=[box_select, "reset"],  # Note this syntax
            title="Select Below",
            x_range=Range1d(start=0.0, end=1.0),
            y_range=Range1d(start=0.0, end=1.0))

rect = Rect(x='x',
            y='y',
            width='width',
            height='height',
            fill_alpha=0.3,
            fill_color='#009933')

p.add_glyph(source, rect, selection_glyph=rect, nonselection_glyph=rect)
show(p)

**Question**: How would you allow the user to erase the glyphs?

In [ ]:
x = [random() for x in range(500)]
y = [random() for y in range(500)]
color = ["navy"] * len(x)

s = ColumnDataSource(data=dict(x=x, y=y, color=color))
p = Figure(plot_width=400, plot_height=400, tools="lasso_select, reset", title="Select Here")
p.circle('x', 'y', color='color', size=8, source=s, alpha=0.4)

s2 = ColumnDataSource(data=dict(x=[0, 1], ym=[0.5, 0.5]))
p.line(x='x', y='ym', color="orange", line_width=5, alpha=0.6, source=s2)

s.callback = CustomJS(args=dict(s2=s2), code="""
        var inds = cb_obj.selected['1d'].indices;
        var d = cb_obj.data;
        var ym = 0

        if (inds.length == 0) { return; }

        for (i = 0; i < d['color'].length; i++) {
            d['color'][i] = "navy"
        }
        for (i = 0; i < inds.length; i++) {
            d['color'][inds[i]] = "firebrick"
            ym += d['y'][inds[i]]
        }

        ym /= inds.length
        s2.data['ym'] = [ym, ym]

        cb_obj.trigger('change');
        s2.trigger('change');
    """)

show(p)

In [ ]:
p = Figure(plot_width=400, plot_height=400,
           tools="tap", title="Click the Dots")

source = ColumnDataSource(data=dict(
    x=[1, 2, 3, 4, 5],
    y=[2, 5, 8, 2, 7],
    color=["navy", "orange", "olive", "firebrick", "gold"]
    ))

p.circle('x', 'y', color='color', size=20, source=source)

url = "http://www.colors.commutercreative.com/@color/"
taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)

show(p)

## Folium

GitHub: https://github.com/python-visualization/folium

Doc: https://folium.readthedocs.io/en/latest/

In [ ]:
import folium
map = folium.Map(location=[38.9071923, -77.0368707])
map

In [ ]:
import geocoder
location = 'King Arthur\'s Seat'
loc = geocoder.google(location)
loc.latlng

In [ ]:
map = folium.Map(location=loc.latlng, zoom_start=15)
folium.Marker(loc.latlng, popup=location).add_to(map)
#folium.CircleMarker(loc.latlng, popup=location, radius=15, color='#3186cc', fill_color='#3186cc').add_to(map)
map

**Exercise**: You can add markers to the map programmatically. Try using requests to get lat/long data from an API or adding the location of tweets as they come in.

In [ ]:
# Extract HTML for embedding
# map.save('map.html')

*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*